In [1]:
import PySimpleGUI as sg
import os.path
import numpy as np
from PIL import ImageFilter
import PIL.Image
import io
import base64
import cv2
from PIL import Image

sg.theme('Purple')

def convert_to_bytes(file_or_bytes, resize=None, operation="", saving="save",path=""):
    if isinstance(file_or_bytes, str):
        img = PIL.Image.open(file_or_bytes)
    else:
        try:
            img = PIL.Image.open(io.BytesIO(base64.b64decode(file_or_bytes)))
        except Exception as e:
            dataBytesIO = io.BytesIO(file_or_bytes)
            img = PIL.Image.open(dataBytesIO)

    cur_width, cur_height = img.size
    if resize:
        new_width, new_height = resize
        scale = min(new_height/cur_height, new_width/cur_width)
        img = img.resize((int(cur_width*scale), int(cur_height*scale)), PIL.Image.ANTIALIAS)
    
    #rotating the image to 90 degrees
    if operation=='rotate':
        img=img.rotate(90)
        if saving=="save":
            img.save("rotated.PNG")  
        
        
    #detecting the edges of the image
    if operation=='edge_detection':
        img= img.filter(ImageFilter.FIND_EDGES)
        if saving=="save":
            img.save("edge.PNG")
    
    #flipping the image with vertical line reference from left to right    
    if operation=='vertical_flip':
        img=img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
        if saving=="save":
            img.save("flip.PNG")
        
   #converting the image to grayscale
    if operation=='grayscale':
        img=img.convert('L')
        if saving=="save":
            img.save("gray.PNG")
        
    #setting the image to red from the rgb format
    if operation=='red':
        img_data=np.array(img)
        img_chn_red = np.zeros(img_data.shape, dtype='uint8')
        img_chn_red[:,:,0] = img_data[:,:,0]
        img= PIL.Image.fromarray(img_chn_red)
        if saving=="save":
            img.save("red.PNG")
            
    #setting the image to green from the rgb format
    if operation=='green':
        img_data=np.array(img)
        img_chn_red = np.zeros(img_data.shape, dtype='uint8')
        img_chn_red[:,:,1] = img_data[:,:,1]
        img= PIL.Image.fromarray(img_chn_red)
        if saving=="save":
            img.save("green.PNG")
        
        
    #setting the image to blue from the rgb format
    if operation=='blue':
        img_data=np.array(img)
        img_chn_red = np.zeros(img_data.shape, dtype='uint8')
        img_chn_red[:,:,2] = img_data[:,:,2]
        img= PIL.Image.fromarray(img_chn_red)
        if saving=="save":
            img.save("blue.PNG")
        
    #reversing the colours of the image
    if operation=='reverse':
        img_data=np.array(img)
        img_reversed_data = 255 - img_data
        img = PIL.Image.fromarray(img_reversed_data)
        if saving=="save":
            img.save("reverse.PNG")
   
    bio = io.BytesIO()
    img.save(bio, format="PNG")
    return bio.getvalue()


file_list_column = [
                    [sg.Text("Image Folder"),
                    sg.In(size=(25, 1), enable_events=True, key="folder"),
                    sg.FolderBrowse()],
                    [sg.Listbox(values=[], enable_events=True, size=(42, 20), key="filelist")],
                    [sg.Button('Save',size=(8,2),key='save',font='sfprodisplay 10 bold',button_color=('black','#D7DBDD'))],
]

image_viewer_column = [
                        [sg.Text("Choose an image from the list displayed:",size=(40, 1), key="imagetext")],
                        [sg.Image(key="image")]
                      ]

button_column=[
    [sg.Button("Resize",size=(10,2),font='sfprodisplay 10 bold',button_color=('#7D3C98', 'white')),
        sg.In(key='width', size=(5,1)), sg.In(key='height', size=(5,1)),
     
    ],
    
    [sg.Button("Rotate",size=(10,2),font='sfprodisplay 10 bold',button_color=('#7D3C98', 'white')),
    sg.Button("Edge detection",size=(8,2),font='sfprodisplay 10 bold',button_color=('#7D3C98', 'white')),
    sg.Button("Vertical flip",size=(10,2),font='sfprodisplay 10 bold',button_color=('#7D3C98', 'white')),
    sg.Button("Grayscale",size=(10,2),font='sfprodisplay 10 bold',button_color=('#7D3C98', 'white')),
    sg.Button("Reverse colour",size=(8,2),font='sfprodisplay 10 bold',button_color=('#7D3C98', 'white'))],
    
    [sg.Button("Image formatting",size=(10,2),font='sfprodisplay 10 bold',button_color=('#7D3C98','white')),
    sg.Text("->"),
    sg.Button("Red",size=(8,2),button_color=('white', 'red'),font='sfprodisplay 10 bold'),
    sg.Button("Green",size=(8,2),button_color=('white', 'green'),font='sfprodisplay 10 bold'),
    sg.Button("Blue",size=(8,2),button_color=('white', 'blue'),font='sfprodisplay 10 bold')]
]

menu_def = [
            ['File', [ 'Exit'  ]]
           ]  

layout = [
        [sg.Menu(menu_def)],
        [sg.Text("Welcome to PixEdit!",font='sfprodisplay 25 bold',justification='C')],
        [sg.Column(file_list_column),
        sg.VSeperator(),
        sg.Column(image_viewer_column)],
        [sg.Text('_'*125) ],
        [sg.Column(button_column)]        
]



window = sg.Window('PixEdit', layout)


while True:
    event, values = window.read()
    if event in (sg.WIN_CLOSED, 'Exit'):
        break
    if event == sg.WIN_CLOSED or event == 'Exit':
        break

             
    #choosing the folder and collecting the files 
    if event == 'folder':                         
        folder = values['folder']
        try:
            file_list = os.listdir(folder)        
        except:
            file_list = []
        fnames = [f for f in file_list if os.path.isfile(
            os.path.join(folder, f)) and f.lower().endswith((".png", ".jpg", "jpeg", ".tiff", ".bmp"))]
        window['filelist'].update(fnames)
     
    #mentioning the resizing dimensions
    elif event == 'filelist':   
        try:
            filename = os.path.join(values['folder'], values['filelist'][0])
            window['imagetext'].update(filename)
            if values['width'] and values['height']:
                new_size = int(values['width']), int(values['height'])
            else:
                new_size = None
            window['image'].update(data=convert_to_bytes(filename, resize=new_size))
        except Exception as E:
            print(f'** Error {E} **')
            pass    
           
    elif event=='Save':
        try:
            filename = os.path.join(values['folder'], values['filelist'][0])
            window['imagetext'].update(filename)
            convert_to_bytes(filename, saving='save')  
        except Exception as E:
            print(f'** Error {E} **')
            pass 
        

            
    #passing the image to rotate
    elif event=='Rotate':
        try:
            filename = os.path.join(values['folder'], values['filelist'][0])
            window['imagetext'].update(filename)
            window['image'].update(data=convert_to_bytes(filename, operation='rotate'))   
        except Exception as E:
            print(f'** Error {E} **')
            pass 
        
        
    elif event=='Edge detection':
        try:
            filename = os.path.join(values['folder'], values['filelist'][0])
            window['imagetext'].update(filename)
            window['image'].update(data=convert_to_bytes(filename, operation='edge_detection'))   
        except Exception as E:
            print(f'** Error {E} **')
            pass 
        
        
    elif event=='Vertical flip':
        try:
            filename = os.path.join(values['folder'], values['filelist'][0])
            window['imagetext'].update(filename)
            window['image'].update(data=convert_to_bytes(filename, operation='vertical_flip'))   
        except Exception as E:
            print(f'** Error {E} **')
            pass 
        
        
    elif event=='Grayscale':
        try:
            filename = os.path.join(values['folder'], values['filelist'][0])
            window['imagetext'].update(filename)
            window['image'].update(data=convert_to_bytes(filename, operation='grayscale'))   
        except Exception as E:
            print(f'** Error {E} **')
            pass 
        
        
    elif event=='Red':
        try:
            filename = os.path.join(values['folder'], values['filelist'][0])
            window['imagetext'].update(filename)
            window['image'].update(data=convert_to_bytes(filename, operation='red'))   
        except Exception as E:
            print(f'** Error {E} **')
            pass 
        
        
    elif event=='Green':
        try:
            filename = os.path.join(values['folder'], values['filelist'][0])
            window['imagetext'].update(filename)
            window['image'].update(data=convert_to_bytes(filename, operation='green'))   
        except Exception as E:
            print(f'** Error {E} **')
            pass 
    
    elif event=='Blue':
        try:
            filename = os.path.join(values['folder'], values['filelist'][0])
            window['imagetext'].update(filename)
            window['image'].update(data=convert_to_bytes(filename, operation='blue'))   
        except Exception as E:
            print(f'** Error {E} **')
            pass 
        
        
    elif event=='Reverse colour':
        try:
            filename = os.path.join(values['folder'], values['filelist'][0])
            window['imagetext'].update(filename)
            window['image'].update(data=convert_to_bytes(filename, operation='reverse'))
        except Exception as E:
            print(f'** Error {E} **')
            pass 
    
    
    
window.close()